# The Domestic Movie Box Office Before, After, and During the COVID-19 Pandemic
#### *Author*: Austin Villegas
#### *Last Updated*: 05/27/2023

# Table of Contents

1. [Problem Statement](#problem_statement)
    * [Definitions](#definitions)
    * [Goals](#goals)
2. [Data Cleaning](#data_cleaning)
    * [Imports](#imports)
    * [Removing Non-Numeric Values and Setting the Data Types](#fix_data_types)
    * [Imputation](#imputation)
3. [Data Exploration](#data_exploration)
    * [Test for Normality](#test_for_normality)
4. [Analysis](#analysis)
    * [z-Test vs. t-Test](#z_test_vs_t_test)
    * [Confidence Intervals](#confidence_intervals)
    * [Analysis of Variance (ANOVA)](#anova)  
5. [Conclusion](#conclusion)

# Problem Statement 
<a id="problem_statement"></a> 

<a id="definitions"></a> 
### Definitions
* `date`: the date the box office gross was collected
* `title`: the title of the movie
* `daily_gross`: the box office gross for the current date
* `num_theaters`: the number of theaters the movie was in	
* `avg_gross`: the average gross per movie theater
* `gross_to_date`: the total gross for the movie up to the current date
* `day_of_release`: the number of days into the release of the movie	
* `distributor`: the distributor for the movie

<a id="goals"></a> 
### Goals

<a id="imports"></a> 
### Imports

In [1]:
# general
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# visualization
import matplotlib.pyplot as plt 

# other
from tqdm.auto import tqdm
import re

# warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

<a id="data_cleaning"></a> 
### Data Cleaning

<a id="fix_data_types"></a> 
#### Removing Non-Numeric Values and Setting the Data Types

In [2]:
# read csv 
df = pd.read_csv('..\\bo_data\\bo_data_daily.csv', sep=',', header=0)
df.head()

,date,title,daily_gross,num_theaters,avg_gross,gross_to_date,day_of_release,distributor
0,2000-01-01,Stuart Little,"$6,762,000","2,900","$2,331","$75,194,370",16,Sony Pictures Entertainment (SPE)
1,2000-01-01,Any Given Sunday,"$6,209,000","2,505","$2,478","$42,404,267",11,Warner Bros.
2,2000-01-01,The Talented Mr. Ripley,"$6,136,000","2,309","$2,657","$35,707,126",8,Paramount Pictures
3,2000-01-01,The Green Mile,"$5,849,000","2,875","$2,034","$73,352,123",23,Warner Bros.
4,2000-01-01,Toy Story 2,"$4,974,364","3,102","$1,603","$204,817,418",44,Walt Disney Studios Motion Pictures


In [3]:
# remove the non-numeric characters and fill in '-' with NaN

# if the movie premiered earlier than the official day of release, day_of_release might be a '-'
# to fix this we can replace '-' with 0 if the daily_gross == gross_to_date (this would mean early release)
df.loc[(df.day_of_release == '-') & (df.daily_gross == df.gross_to_date), 'day_of_release'] = 0

# remove all '-' in other columns and replace with None
df.daily_gross = df.daily_gross.replace('-', np.NaN)
df.num_theaters = df.num_theaters.replace('-', np.NaN)
df.avg_gross = df.avg_gross.replace('-', np.NaN)
df.gross_to_date = df.gross_to_date.replace('-', np.NaN)
df.day_of_release = df.day_of_release.replace('-', np.NaN)

# remove all non-numeric symbols from numeric fields like '$' or ','
regex = re.compile(r'[^0-9.]')
df.daily_gross = df.daily_gross.str.replace(regex, '', regex=True)
df.num_theaters = df.num_theaters.str.replace(regex, '', regex=True)
df.avg_gross = df.avg_gross.str.replace(regex, '', regex=True)
df.gross_to_date = df.gross_to_date.str.replace(regex, '', regex=True)
df.day_of_release = df.day_of_release.str.replace(regex, '', regex=True)

# right now all objects in df are str, need to convert to correct type
type_dict = { 
    'title' : str,
    'daily_gross' : float,
    'num_theaters' : float,
    'avg_gross' : float,
    'gross_to_date' : float,
    'day_of_release' : float,
    'distributor' : str  
}

df = df.astype(type_dict)
df.date = pd.to_datetime(df.date, format='%Y/%m/%d')

In [4]:
print('total nulls:', len(df[df.isnull().any(axis=1)]) + len(df[df.distributor == '-']))
print('nulls from distributor:', len(df[df.distributor == '-']))
print('numeric nulls:', len(df[df.isnull().any(axis=1)]))

total nulls: 21972
nulls from distributor: 21659
numeric nulls: 313


<a id="imputation"></a> 
#### Imputation

We need to impute data for num_theaters, avg_gross, gross_to_date, and distributor. The amount of numeric nulls is quite low relative to the total amount of data, and normally I would just remove these rows. But because of the use case, I believe these values are easy and worthy to impute. Imputing distributors will not be as fruitful, but I can try to fill in some of the gaps by seeing if the row just has a missed entry and the distributor for that movie exists in another row.

The rules for imputation will be as follows:
1. **num_theaters**
    * We will impute using the same values as the closest date with a known value. In the event that two dates are equally close we will take the minimum between the two values.
    * If the value is only a single day with no other surrounding days we can approximate num_theaters as daily_gross/$500 (with a $10 avg price this translates to 50 people per theater)
2. **avg_gross**
    * Once we impute num theaters we can calculate avg_gross as daily_gross/num_theaters
3. **gross_to_date**
    * If it is the first or zeroth day of release we can set this values to the daily_gross. If not the first day we can perform the addition by hand.
4. **distributor**
    * We can check to see if the distributor exists for the movie in any another row. If it does not, we will leave the value as '-'.

In [5]:
# impute num_theaters

# get list of movies with Null num_theaters
movies_null = df[df.num_theaters.isna()].title.unique()

for movie in movies_null:
    # get the list of dates with Null num_theaters
    dates_null = df[(df.title == movie) & (df.num_theaters.isna())].date

    # make a list of all valid dates and num_theaters
    dates_valid = df.loc[(df.title == movie) & (df.num_theaters.notna())].date
    num_valid = np.array(df.loc[(df.title == movie) & (df.num_theaters.notna())].num_theaters)

    # if single-day release or if no number_theaters were tracked for entire release
    if (len(df[(df.title == movie)]) == 1) | (len(dates_valid) == 0):
        df.loc[(df.title == movie), 'num_theaters'] = round(df.loc[(df.title == movie), 'daily_gross'].mean()/500)
        continue

    # for every null date
    for d1 in dates_null:
        # find the closest dates and values
        diff_list = np.array([abs((d1 - d2).days) for d2 in dates_valid])
        min_indxs = np.where(diff_list == diff_list.min())

        if len(min_indxs) == 1: 
            min_value = num_valid[min_indxs[0]][0]

        else:
            min_value = np.minimum(num_valid[min_indxs[0]], num_valid[min_indxs[1]])

        df.loc[(df.title == movie) & (df.date == d1), 'num_theaters'] = min_value

In [6]:
# impute avg_gross
df.loc[df.avg_gross.isna(), 'avg_gross'] = round(df[df.avg_gross.isna()].daily_gross / df[df.avg_gross.isna()].num_theaters)

In [7]:
# impute gross_to_date

# get list of movies with Null gross_to_date
movies_null = df[df.gross_to_date.isna()].title

for movie in movies_null:
    # if only one date
    if len(df[df.title == movie]) == 1:
        df.loc[(df.title == movie), 'gross_to_date'] = df.loc[(df.title == movie), 'daily_gross']
        continue

    for i in range(len(df[df.title == movie])-1, 0, -1):
        if df[df.title == movie].iloc[i-1, df.columns.get_loc('gross_to_date')] != df[df.title == movie].iloc[i-1, df.columns.get_loc('gross_to_date')]:
            indx = df[df.title == movie].index[i-1]
            indx_1 = df[df.title == movie].index[i]
            df.loc[indx, 'gross_to_date'] = df.loc[indx_1, 'gross_to_date'] - df.loc[indx_1, 'daily_gross']

In [8]:
# impute distributor

# get list of movies with Null distributor
movies_null = df[df.distributor == '-'].title.unique()

for movie in movies_null:
    # movie has a distributor somewhere
    dist = df[df.title == movie].distributor.unique()
    if len(dist) > 1:
        if dist[0] == '-':
            df.loc[(df.title == movie), 'distributor'] = dist[1]
        else:
            df.loc[(df.title == movie), 'distributor'] = dist[0]

<a id="data_exploration"></a> 
# Data Exploration

<a id="test_for_normality"></a> 
### Test for Normality

<a id="analysis"></a> 
# Analysis 

<a id="z_test_vs_t_test"></a> 
### z-Test vs t-Test

<a id="confidence_intervals"></a> 
### Confidence Intervals

<a id="anova"></a> 
### Analysis of Variance (ANOVA)

<a id="conclusion"></a> 
# Conclusion 